In [58]:
import pandas as pd
import numpy as np
from collections import Counter

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error

In [33]:
data = pd.read_csv('CLEAN.csv',index_col=0)
# print(data['misarticulation_index'].values[:15])
data = data.loc[data['first_lang_english']==1]
data.drop(columns=['first_lang_english'],inplace=True)
data = data[['response_text','misarticulation_index']]

data['misarticulation_index'] = (data['misarticulation_index']/0.33333).astype(int).astype(float)
# print(data['misarticulation_index'].values[:15])

In [34]:
for text in tqdm(data['response_text']):
    print(text)

100%|██████████████████████████████████████| 504/504 [00:00<00:00, 41210.41it/s]

In the internship I just finished, I had to write rejection letters, editorial letters, and reader's reports. These documents require you to be very direct with writers, sharing the status of their submitted manuscript and, often, what changes you would like to see them make. My difficulty was that I struggled to be direct with the writers, even if they were hypothetical. I didn't want to be cruel or judgmental. So I met with the internship director, who challenged me to try to learn into judgment even though it was uncomfortable. What I found in following his instructions was that even when I felt like I was being mean, I was just producing a much more direct and helpful document than usual. So even though I'm still working on this, ultimately I learned how that direct communication is kind communication.
As I'm in college to become a preschool teacher, something that's very important to me is always having compassion and understanding towards others.
Dear Soon-to-be Parent,
Congratul

In [38]:
words = []
for text in tqdm(data['response_text']):
    t_tokens = nltk.word_tokenize(text)
    words.extend(t_tokens)

100%|███████████████████████████████████████| 504/504 [00:00<00:00, 2243.40it/s]


In [39]:
word_set = set(words)

In [40]:
len(word_set)

4295

In [45]:
# https://nlp.stanford.edu/projects/glove/
word_embs = []
with open('glove.42B.300d.txt','r',encoding='utf-8') as fr:
    for line in tqdm(fr):
        line = line.strip().split(' ')
        word = line[0]
        if word in word_set:
            word_embs.append(line)

1917495it [00:26, 72423.55it/s]


In [46]:
word_embs = pd.DataFrame(word_embs).set_index(0).astype(float)

In [47]:
word_embs.shape

(3800, 300)

In [48]:
word_embs.head(5)

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
",",0.18378,-0.121230,-0.119870,0.015227,-0.191210,-0.066074,-2.9876,0.807950,0.067338,-0.131840,...,0.136150,0.109990,-0.334740,-0.046109,0.10780,-0.035657,-0.012921,-0.039038,0.182740,0.146540
the,-0.20838,-0.149320,-0.017528,-0.028432,-0.060104,-0.264600,-4.1445,0.629320,0.336720,-0.433950,...,-0.041244,-0.461820,0.027903,0.546570,-0.25894,0.395150,0.261440,-0.540660,0.211990,-0.009436
.,0.10876,0.002244,0.222130,-0.121020,-0.048959,0.018135,-3.8174,-0.032631,-0.625940,-0.518980,...,0.063131,-0.212300,-0.300880,-0.451610,0.26480,0.075971,-0.406880,-0.296960,0.159390,-0.149020
and,-0.09611,-0.257880,-0.358600,-0.328870,0.579500,-0.517740,-4.1582,-0.113710,-0.108480,-0.488850,...,0.477810,-0.021213,-0.212360,0.423740,0.14083,0.067498,-0.126750,-0.370300,-0.092774,0.390580
to,-0.24837,-0.454610,0.039227,-0.284220,-0.031852,0.263550,-4.6323,0.013890,-0.539280,-0.084454,...,0.082736,-0.624690,0.044267,0.606730,-0.12458,-0.154430,-0.163390,0.053097,0.154580,-0.380530


In [49]:
valid_word_set = set(word_embs.index)

In [50]:
text_embs = []
for text in tqdm(data['response_text']):
    emb = word_embs.loc[list(filter(lambda x:x in valid_word_set,nltk.word_tokenize(text)))].mean(0).values
    text_embs.append(emb)

100%|████████████████████████████████████████| 504/504 [00:00<00:00, 782.23it/s]


In [51]:
# y

In [166]:
X = np.array(text_embs)
y = data['misarticulation_index'].values

split_ = np.random.RandomState(seed=0).permutation(X.shape[0])
num_train = int(X.shape[0]*0.7)
X_train, X_test, y_train, y_test = X[:num_train,:], X[num_train:,:], y[:num_train], y[num_train:]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)

In [167]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(352, 300) (152, 300) (352,) (152,)


In [168]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

regr = svm.SVR(kernel='linear', epsilon=0.9)
regr.fit(X_train, y_train)

SVR(epsilon=0.9, kernel='linear')

In [16]:
#default parameters
ms = [mean_absolute_error, mean_squared_error, median_absolute_error]
for m in ms:
    print(m, m(y_test/3., regr.predict(X_test)/3.))

<function mean_absolute_error at 0x00000205E79E9438> 0.8782875324918789
<function mean_squared_error at 0x00000205E79E9168> 1.1964850734171366
<function median_absolute_error at 0x00000205E79E45E8> 0.6676549555600968


In [169]:
#tuned model

ms = [mean_absolute_error, mean_squared_error, median_absolute_error]
for m in ms:
    print(m, m(y_test/3., regr.predict(X_test)/3.))

<function mean_absolute_error at 0x7f834a0a8940> 0.813298969025324
<function mean_squared_error at 0x7f834a0ae310> 1.1368651953843225
<function median_absolute_error at 0x7f834a0ae550> 0.6281113519667647
